# sfMix reading

We'll use this notebook to help read in sfMix experiments and manage the MountainSort spikes

Last change: 22.02.22   
Last opened: 22.02.22  

In [2]:
import os, sys, glob
import helper_fcns as hf
sys.path.insert(0, 'ExpoAnalysisTools/python/')
from ExpoAnalysisTools.python import read_sfMix, readBasicCharacterization
import numpy as np
import pandas as pd

In [3]:
import importlib as il

# LGN

#### For the LGN, we'll also need to take into account the classification (i.e. Magno/Parvo/Other). We can use Manu's excel sheet to do this.

In [4]:
basePath = os.getcwd() + '/'
sheetName = 'manu-classification-220222.xlsx';
# sheetName = 'manu-classification.xlsx'; # original
# There are 3 sheets - m675, m680, m682
readIn = pd.read_excel(basePath + 'LGN/experiments/' + sheetName, sheet_name=['m675', 'm680', 'm682'])
def idToStr(typeId):
    if int(typeId) == 0:
        return 'parvo';
    elif int(typeId) == 1:
        return 'magno';
    else:
        return 'tbd'; # i.e. Manu says he couldn't classify definitively
    
readInFlat = pd.concat(readIn, keys=['m675', 'm680', 'm682']).reset_index()
# rename some things
readInFlat.rename(columns={'level_0': 'm#', 'level_1': 'expCount',
                          'celllabel': 'label', 'cellno': 'cellNum',
                          'expono': 'progNum'}, inplace=True)

# Are there NaN types?
nans = np.where(np.isnan(readInFlat['type']))[0]
readInFlat['label'][nans]

12    r22#5
18    r32#5
19    r33#5
Name: label, dtype: object

In [5]:
# manually edit the NaN types...
readInFlat.at[12, 'type'] = 0 # parvo, per notes
readInFlat.at[18, 'type'] = 2 # TBD, per notes
readInFlat.at[19, 'type'] = 2 # TBD, per notes

In [6]:
# Convert the index into a string with the type
readInFlat['typeStr'] = readInFlat['type'].apply(idToStr)
readInFlat['fullName'] = ['%s%s' % (x,y) for x,y in zip(readInFlat['m#'], readInFlat['label'])]

So, now we have a dataframe with the eccentricity and type for each recorded LGN cell. We can use this in the creation of the datalist to ensure that each cell has an associated eccentricity and type that will carry through all of the analyses.

### Convert the .xml to .npy!

In [39]:
il.reload(read_sfMix);

In [40]:
dl_name_read = 'dataList_210524.npy'
dl_name_save = 'dataList_220222.npy'
read_sfMix.read_sfMix_all('LGN/recordings/', dl_name=dl_name_save)

Analyzing LGN/recordings/m675r04#4[sfMixLGN].xml
saving: LGN/structures/m675r04_sfm.npy
Analyzing LGN/recordings/m675r06#6[sfMixLGN].xml
saving: LGN/structures/m675r06_sfm.npy
Analyzing LGN/recordings/m675r08#5[sfMixLGN].xml
saving: LGN/structures/m675r08_sfm.npy
Analyzing LGN/recordings/m675r13#4[sfMixLGN].xml
saving: LGN/structures/m675r13_sfm.npy
Analyzing LGN/recordings/m675r15#11[sfMixLGN].xml
Analyzing LGN/recordings/m675r15#7[sfMixLGN].xml
saving: LGN/structures/m675r15_sfm.npy
Analyzing LGN/recordings/m675r16#7[sfMixLGN].xml
saving: LGN/structures/m675r16_sfm.npy
Analyzing LGN/recordings/m675r17#5[sfMixLGN].xml
saving: LGN/structures/m675r17_sfm.npy
Analyzing LGN/recordings/m675r18#6[sfMixLGN].xml
saving: LGN/structures/m675r18_sfm.npy
Analyzing LGN/recordings/m675r19#6[sfMixLGN].xml
saving: LGN/structures/m675r19_sfm.npy
Analyzing LGN/recordings/m675r20#6[sfMixLGN].xml
saving: LGN/structures/m675r20_sfm.npy
Analyzing LGN/recordings/m675r21#5[sfMixLGN].xml
saving: LGN/structure

### Update the dataList
Remember that the previous function above creates/updates a dataList

In [41]:
# load the dataList
dl = hf.np_smart_load('LGN/structures/%s' % dl_name_save)
cellType = np.nan * np.zeros((len(dl['unitName'], )), dtype='O')

In [42]:
# Now, go through and find (in the readInFlat dataframe) the corresponding cell type, if there
for ii, nm in enumerate(dl['unitName']):
    split = nm.split('l');
    if len(split) == 1: # i.e. there was NOT an l in this name...
        
        mStr, unitNum = hf.parse_exp_name(nm)[0:2] # first two arguments
        
        validRow = np.where((readInFlat['m#'] == mStr) & (readInFlat['cellNum'] == unitNum))[0]
        if len(validRow)>0:
            cellType[ii] = readInFlat.at[validRow[0], 'typeStr']
        else:
            print('No match for %s' % nm)
    

dl['unitType'] = cellType
np.save('LGN/structures/%s' % dl_name_save, dl)

In [43]:
for nm in ['parvo', 'magno', 'tbd']:
    ok = len(np.where(dl['unitType'] == nm)[0])
    print('%d %s cells' % (ok, nm))

43 parvo cells
27 magno cells
11 tbd cells


#### Copy over the basic program information from the reference file dataList (210524)

In [45]:
dl = hf.np_smart_load('LGN/structures/%s' % dl_name_save)
dl_og = hf.np_smart_load('LGN/structures/%s' % dl_name_read);

##### Make sure all the names are aligned before proceeding

In [46]:
['%s -- %s' % (x,y) for x,y in zip(dl['unitName'], dl_og['unitName'])]

['m675r04 -- m675r04',
 'm675r06 -- m675r06',
 'm675r08 -- m675r08',
 'm675r13 -- m675r13',
 'm675r15 -- m675r15',
 'm675r16 -- m675r16',
 'm675r17 -- m675r17',
 'm675r18 -- m675r18',
 'm675r19 -- m675r19',
 'm675r20 -- m675r20',
 'm675r21 -- m675r21',
 'm675r22 -- m675r22',
 'm675r23 -- m675r23',
 'm675r24 -- m675r24',
 'm675r26 -- m675r26',
 'm675r27 -- m675r27',
 'm675r32 -- m675r32',
 'm675r33 -- m675r33',
 'm675r36 -- m675r36',
 'm675r37 -- m675r37',
 'm675r38 -- m675r38',
 'm675r39 -- m675r39',
 'm675r40 -- m675r40',
 'm675r42 -- m675r42',
 'm675r43 -- m675r43',
 'm675r45 -- m675r45',
 'm675r47 -- m675r47',
 'm675r48 -- m675r48',
 'm675r49 -- m675r49',
 'm675r50 -- m675r50',
 'm675r51 -- m675r51',
 'm675r52 -- m675r52',
 'm675r54 -- m675r54',
 'm675r56 -- m675r56',
 'm680p01r03 -- m680p01r03',
 'm680p01r04 -- m680p01r04',
 'm680p01r05 -- m680p01r05',
 'm680p01r06 -- m680p01r06',
 'm680p01r07 -- m680p01r07',
 'm680p01r09 -- m680p01r09',
 'm680p01r10 -- m680p01r10',
 'm680p01r12 --

In [48]:
dl['basicProgName'] = dl_og['basicProgName']
dl['basicProgOrder'] = dl_og['basicProgOrder']

In [49]:
np.save('LGN/structures/%s' % dl_name_save, dl)

# V1

In [151]:
il.reload(read_sfMix);

In [152]:
dl_name = 'dataList_210528.npy'
# read_sfMix.read_sfMix_all('V1/recordings/', dl_name=dl_name)

In [153]:
test_files = glob.glob('V1/structures/m*_sfm.npy')

unitNames = [];
expTypes = [];
unitArea = [];

for t in sorted(test_files):
    
    curr_file = hf.np_smart_load(t);
    
    unitNames.append(curr_file['unitLabel'])
    expName = curr_file['sfm']['exp']['filename'];
    expTypes.append(hf.parse_exp_name(expName)[-2]);
    unitArea.append('V1')
    
dataList = dict();
dataList['unitName'] = unitNames;
dataList['expType'] = expTypes
dataList['unitArea'] = unitArea;

print('%d cells' % len(unitNames))

np.save('V1/structures/%s' % dl_name, dataList)

11 cells
